In [ ]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\PRJ_project\LAquilaReducedScale.tgt'
output_folder = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\SI_conversion_validation'

file_path = Path(file_path)
tgtA = LTFdb()
tgtA.read(file_path)

# Make sure output directory exists
out_dir = Path(output_folder)
out_dir.mkdir(parents=True, exist_ok=True)

# Determine output filename
file_path = Path(file_path)
filename = file_path.name + "SimpleFunc.txt"
output_path = out_dir / filename

# Write the .ltf file into the specified folder
tgtA.write_txt(str(output_path))

tgtA.write_txt(str(output_path))

In [ ]:
# Python standard library
import os
import sys
import datetime

from pathlib import Path
# pxc modules
if (ppfolder := str(Path('..') / 'personal_python_packages')) not in sys.path:
    sys.path.append(ppfolder)

import numpy as np
from scipy.constants import g

from utilities import maxabs
from LNECSPA import LTFdb

def ltf_to_txt(file_path):
    file_path = Path(file_path)
    tgtA = LTFdb()
    tgtA.read(file_path)

    # 1. Identify channel indices
    disp_inds = [i for i, unit in enumerate(tgtA.units) if unit.lower() in ('mm', 'm')]
    acc_inds  = [i for i, typ  in enumerate(tgtA.types) if typ.lower() == 'acceleration']

    # 2. Build time vector
    dt     = float(tgtA.dt[0])
    n_pts  = tgtA.data.shape[1]
    time_s = np.arange(n_pts) * dt

    # 3. Convert to SI
    # Displacement channels
    disp_si = tgtA.data[disp_inds].astype(float)
    for idx, ch in enumerate(disp_inds):
        if tgtA.units[ch].lower() == 'mm':
            disp_si[idx] *= 1e-3

    # Acceleration channels
    acc_si = tgtA.data[acc_inds].astype(float)
    for idx, ch in enumerate(acc_inds):
        if tgtA.units[ch].lower() == 'g':
            acc_si[idx] *= g

    # Print min/max for each channel
    print("Displacement channels (SI units):")
    for idx, ch in enumerate(disp_inds):
        name = tgtA.names[ch]
        arr = disp_si[idx]
        print(f"  {name}: min = {arr.min():.6e}, max = {arr.max():.6e}")

    print("Acceleration channels (m/s^2):")
    for idx, ch in enumerate(acc_inds):
        name = tgtA.names[ch]
        arr = acc_si[idx]
        print(f"  {name}: min = {arr.min():.6e}, max = {arr.max():.6e}")

    # 4. Stack columns: time | displacements | accelerations
    out_arr = np.vstack((time_s, disp_si, acc_si)).T

    # 5. Remove columns that are entirely zero
    mask = ~np.all(np.isclose(out_arr, 0.0), axis=0)
    out_arr = out_arr[:, mask]

    # 6. Prepare headers aligned with filtered columns
    header_cols = ['time_s'] + \
        [f"{tgtA.names[i]}_m" for i in disp_inds] + \
        [f"{tgtA.names[i]}_m_per_s2" for i in acc_inds]
    header_cols = [h for h, m in zip(header_cols, mask) if m]

    # 7. Export using tabs as separator, full precision (default)
    ext = file_path.suffix.lstrip('.')
    out_name = f"{file_path.stem}_{ext}.txt"
    out_path = Path.cwd() / out_name

    header_line = '\t'.join(header_cols)
    np.savetxt(
        out_path,
        out_arr,
        delimiter='\t',
        header=header_line,
        comments=''
    )#fmt='%.18e',  # high precision scientific format

    print(f"Exported {out_arr.shape[0]} samples and {out_arr.shape[1]} columns to '{out_path}'.")



In [13]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\LAquilaReducedScale.tgt'

ltf_to_txt(file_path)

Displacement channels (SI units):
  DispT: min = -2.271209e-02, max = 1.616917e-02
  DispL: min = 0.000000e+00, max = 0.000000e+00
  DispV: min = 0.000000e+00, max = 0.000000e+00
Acceleration channels (m/s^2):
  AccT: min = -6.778052e-01, max = 6.183798e-01
  AccL: min = 0.000000e+00, max = 0.000000e+00
  AccV: min = 0.000000e+00, max = 0.000000e+00
Exported 8982 samples and 3 columns to 'c:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\LTF_to_TXT\LAquilaReducedScale_tgt.txt'.


In [14]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.DRV'
ltf_to_txt(file_path)

Displacement channels (SI units):
  DispT: min = -1.359835e-01, max = 9.814143e-02
  DispL: min = 0.000000e+00, max = 0.000000e+00
  DispV: min = 0.000000e+00, max = 0.000000e+00
Acceleration channels (m/s^2):
Exported 8982 samples and 2 columns to 'c:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\LTF_to_TXT\LAquilaReducedScale_34_DRV.txt'.


In [15]:
file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34.acq'
ltf_to_txt(file_path)

Displacement channels (SI units):
  PosA1T: min = -1.442619e-01, max = 1.006543e-01
  PosA2L: min = -3.200000e-05, max = 3.199902e-05
  PosA3V: min = 1.385946e-03, max = 1.584014e-03
Acceleration channels (m/s^2):
Exported 8982 samples and 4 columns to 'c:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\uniaxial_table_model\Adapting_Driver_Signal\LTF_to_TXT\LAquilaReducedScale_34_acq.txt'.


In [16]:
# file_path = r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\LNEC_ERIES_RE-SAFE\CTL\TestLAquila\2024-09-11\LAquilaReducedScale_34_mon.acq'
# ltf_to_txt(file_path)